# Working with Regular Expressions


Сбор информации(*url новости, заголовок новости и ее тематика (тэг)*) о новостях со всех страниц сайта. Эта информация сохраняется в датафрейм, со следующими колонками:
* `url` - ссылка (должна быть полной, т.е. начинаться с http или https) на новость (тип колонки - `str`).
* `title` - заголовок новости (тип колонки - `str`).
* `tag` - категория новости (новости/истории/шапито/...) (тип колонки - `str`, не должен содержать ковычек).

In [ ]:
import requests
import re
import pandas as pd

page_list = []
url = 'http://news-rg.task-sss.krasilnikov.spb.ru/dd2bdf6cb881176bc2c0c568e1a180fa/1'
i = 1
while True:
    page = requests.get(url + str(i))
    if page.status_code == 500:
        continue
    elif page:
        page_list.append(page.text)
        i += 1
    elif page.status_code == 404:
        break

urls = []
titles = []
tags = []
pattern_for_tag = '<a href="" class="b-link b-link_tag">(.*)</a>'
pattern_for_link_and_title = '<a class="b-link b-link_title".*\n.*href="(.+)">.*\n\s*(.+)</a>'
for page in page_list:
    tags.extend(re.findall(pattern_for_tag, page))

    urls_and_titles = re.findall(pattern_for_link_and_title, page)
    urls.extend(['http://news-rg.task-sss.krasilnikov.spb.ru' + item[0] for item in urls_and_titles])
    titles.extend([item[1] for item in urls_and_titles])

news_tag = pd.DataFrame({'url': urls, 'title': titles, 'tag': tags})
news_tag[['url', 'title', 'tag']] = news_tag[['url', 'title', 'tag']].astype(str)
news_tag

,url,title,tag
0,http://news-rg.task-sss.krasilnikov.spb.ru/new...,В Подмосковье для борьбы с коронавирусом вновь...,новости
1,http://news-rg.task-sss.krasilnikov.spb.ru/new...,"«Что-то шипит, трещит и щелкает». Астрономы да...",истории
2,http://news-rg.task-sss.krasilnikov.spb.ru/new...,"На самом деле, на выборах в США победили сторо...",истории
3,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Голосование на выборах президента США завершил...,новости
4,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Половина российских врачей заявила о падении з...,новости
...,...,...,...
85,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Кадыров назвал Путина защитником оппозиции и п...,новости
86,http://news-rg.task-sss.krasilnikov.spb.ru/new...,США в Совбезе ООН пригрозили финансовыми санкц...,новости
87,http://news-rg.task-sss.krasilnikov.spb.ru/new...,К журналистке Tut.by Галине Уласик пришли с об...,новости
88,http://news-rg.task-sss.krasilnikov.spb.ru/new...,ОМОН перекрыл центр Минска перед «Маршем единс...,новости


Для каждой новости со всех страниц сайта собирается информация об изображении в тексте статьи и точное время ее публикации. Данная информация помещается в датафрейм со следующими колонками:

* `url` - ссылка (должна быть полной, т.е. начинаться с http или https) на новость (тип колонки - `object`).
* `title` -  заголовок новости (тип колонки - `object`).
* `pic_url` - URL картинки в тексте статьи. На каждой из них есть логотип "Meduza" (тип колонки - `object`).
* `time` - точное время, когда была опубликована новость в формате `dd.mm.yyyy hh:mm` (тип колонки - `object`).

In [ ]:
def request_page(url):
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            break
    page = response.text
    title = re.findall('<h1 class="b-material-head__title">(.*?)</h1>', page)[0].strip()
    pic_url = re.findall('img style="-webkit-user-select: none;margin: auto;cursor: zoom-in;" src="(.*?)"', page)[0]
    time = " ".join(re.findall('([0-9]{2}\.[0-9]{2}\.[0-9]{4})[\s\S]*?([0-9]{2}:[0-9]{2})', page)[0])
    return {"url": url,
            "title": title,
            "pic_url": pic_url,
            "time": time}

news_pictures = pd.DataFrame(columns=["url", "title", "pic_url", "time"])

for url in news_tag.url:
    news_pictures = news_pictures.append(request_page(url), ignore_index=True)

news_pictures = news_pictures.astype("object")
news_pictures

,url,title,pic_url,time
0,http://news-rg.task-sss.krasilnikov.spb.ru/new...,В Подмосковье для борьбы с коронавирусом вновь...,https:/meduza.io//imgly/share/1604922066/news/...,07.09.2020 18:09
1,http://news-rg.task-sss.krasilnikov.spb.ru/new...,"«Что-то шипит, трещит и щелкает». Астрономы да...",https:/meduza.io//imgly/share/1604655676/featu...,07.09.2020 18:09
2,http://news-rg.task-sss.krasilnikov.spb.ru/new...,"На самом деле, на выборах в США победили сторо...",https:/meduza.io//imgly/share/1604564484/featu...,07.09.2020 18:09
3,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Голосование на выборах президента США завершилось,https:/meduza.io//imgly/share/1604471550/news/...,07.09.2020 18:09
4,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Половина российских врачей заявила о падении з...,https:/meduza.io//imgly/share/1603708938/news/...,07.09.2020 18:09
...,...,...,...,...
85,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Кадыров назвал Путина защитником оппозиции и п...,https:/meduza.io//imgly/share/1601641154/news/...,07.09.2020 18:09
86,http://news-rg.task-sss.krasilnikov.spb.ru/new...,США в Совбезе ООН пригрозили финансовыми санкц...,https:/meduza.io//imgly/share/1599803741/news/...,07.09.2020 18:09
87,http://news-rg.task-sss.krasilnikov.spb.ru/new...,К журналистке Tut.by Галине Уласик пришли с об...,https:/meduza.io//imgly/share/1599636468/news/...,07.09.2020 18:09
88,http://news-rg.task-sss.krasilnikov.spb.ru/new...,ОМОН перекрыл центр Минска перед «Маршем единс...,https:/meduza.io//imgly/share/1599393336/news/...,07.09.2020 18:09


Расчет метрики TF-IDF: метрика важности слова в новости или в любом другом документе. Для каждой новости с первых двух(!) страниц сайта и для каждого слова в текстах этих новостей (без заголовков). Перед расчетом метрики было необходимо:
1) оставить в тексте только буквы и пробельные знаки
2) привести все слова к нижнему регистру
3) привести все слова к начальной форме с помощью библиотеки pymystem3

В переменную `important_words` записывается датафрейм со следующими колонками:
* `title` - заголовок новости (тип колонки `object`)
* `max_tfidf_word` - слово с максимальным значением TF-IDF (тип колонки `object`)
* `max_tfidf` - максимальное значение TF-IDF для каждой новости (тип колонки `float64`)
Новости должны быть в том порядке, в котором они представлены на сайте.



In [ ]:
### important_words=...
from sklearn.feature_extraction.text import TfidfVectorizer

def prepare_page(url):
    resp = rq.get(url)
    while resp.status_code != 200:
        resp = rq.get(url)
        page = resp.text

    text = re.findall('b-material-wrapper__text">([\s\S]*?)</div', page)[0]
    clean_text = text.replace("<p>", " ").replace("</p>", " ")
    clean_text = "".join(re.findall("[\sА-Яа-яA-Za-z]*?", clean_text))

    title = re.findall('class="b-material-head__title">([\s\S]*?)<', page)[0].strip()

    m = Mystem()

    lemmatized = "".join(m.lemmatize(clean_text))
    return lemmatized, title

texts = []
titles = []

for url in all_urls:
    lemmatized, title = prepare_page(url)
    titles.append(title)
    texts.append(lemmatized)

titles

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)
important_words = pd.DataFrame(X.A)
important_words = important_words.rename(columns=dict(zip(list(important_words.columns), vectorizer.get_feature_names())))
important_words["max_tfidf"] = important_words.max(axis=1)
important_words["max_tfidf_word"] = important_words.idxmax(axis=1)
important_words["title"] = titles
important_words = important_words[["title", "max_tfidf", "max_tfidf_word"]]

important_words = important_words.astype({"title": "object", "max_tfidf": "float64", "max_tfidf_word": "object"})
important_words